In [5]:
%load_ext autoreload
%autoreload
%matplotlib inline

from news_crawler import NewsCrawler
from IPython.display import display
from IPython.core.debugger import set_trace

In [6]:
crawler = NewsCrawler(storage='local')

checking local storage... done
we have total 456145 articles (280622 downloaded, 175523 trashed)


# 1. URL을 모은다

In [7]:
# 제거고려: nextbigfuture, nbcnews, joongang, chosun

In [8]:
collect_result = crawler.collect(); display(*collect_result)

100.00% (89.65 seconds): Collecting URLs... yahoo               

100.00% (0.11 seconds): Selecting URLs... cryptonews          

,collected,collected_unique,selected
all,26190,26112,143
reuters,1120,1120,29
yahoo,361,361,25
investing.com,994,994,21
chinadaily,697,697,15
...,...,...,...
huffpost,66,66,0
axios,134,132,0
blastingnews,117,117,0
financialpost,107,107,0


,url,pubs,actual_pub
0,https://politico.com/magazine/story/2019/10/23...,"politico, realclearpolitics",politico
1,https://politico.com/news/2019/10/27/adam-schi...,"politico, realclearpolitics",politico
2,https://politico.com/news/2019/10/28/democrats...,"politico, realclearpolitics",politico
3,https://politico.com/magazine/story/2019/10/23...,"politico, realclearpolitics",politico
4,https://politico.com/magazine/story/2019/10/21...,"politico, realclearpolitics",politico
...,...,...,...
72,https://nbcnews.com/news/us-news/family-inadve...,"msnbc, nbcnews",nbcnews
73,https://nbcnews.com/news/us-news/man-accused-m...,"msnbc, nbcnews",nbcnews
74,https://nbcnews.com/think/opinion/will-dolphin...,"msnbc, nbcnews",nbcnews
75,https://nbcnews.com/feature/nbc-out/tennessee-...,"msnbc, nbcnews",nbcnews


# 2. 해당 URL의 뉴스를 다운로드한다

In [9]:
download_result = crawler.download(); download_result

100.00% (80.59 seconds): Downloading... yahoo               

collect                             download              
              collected collected_unique selected downloaded trashed total
all               26190            26112      143         88      55   143
reuters            1120             1120       29         13      16    29
yahoo               361              361       25         17       8    25
investing.com       994              994       21          9      12    21
chinadaily          697              697       15         13       2    15
...                 ...              ...      ...        ...     ...   ...
huffpost             66               66        0          0       0     0
axios               134              132        0          0       0     0
blastingnews        117              117        0          0       0     0
financialpost       107              107        0          0       0     0
cryptonews          327              327        0          0       0     0

[74 rows x 6 columns]

In [28]:
download_result.to_dict()

{('collect', 'collected'): {'all': 8645,
  'reuters': 764,
  'fox': 299,
  'investing.com': 510,
  'cnn': 623,
  'telegraph': 542,
  'news24': 270,
  'marketwatch': 137,
  'washingtonpost': 137,
  'wsj': 204,
  'aljazeera': 168,
  'thestreet': 128,
  'washingtontimes': 115,
  'nationalreview': 730,
  'guardian': 164,
  'financialpost': 113,
  'msnbc': 136,
  'nbcnews': 87,
  'zdnet': 239,
  'businessinsider': 76,
  'cnbc': 66,
  'bbc': 192,
  'afr': 86,
  'vice': 105,
  'techcrunch': 20,
  'axios': 21,
  'politico': 206,
  'realclearpolitics': 139,
  'nytimes': 99,
  'theverge': 947,
  'scmp': 252,
  'thehill': 88,
  'huffpost': 65,
  'chinadaily': 393,
  'morningstar': 32,
  'rt': 16,
  'indiatimes': 33,
  'cryptonews': 327,
  'time': 19,
  'japan-news': 70,
  'townhall': 0,
  'arirang': 0,
  'yahoo': 16,
  'americanconservative': 0,
  'atlantic': 0,
  'arynews': 0,
  'usatoday': 0,
  'thenextweb': 0,
  'nymag': 0,
  'thinkprogress': 0,
  'arstechnica': 0,
  'newrepublic': 0,
  'npr':

In [13]:
10000/50

200.0

Bigquery와 local 뉴스 갯수가 다를 것이다. Bigquery에 뉴스를 모두 업데이트 한 후 해당 repository에 문제가 생겨서, github에서 전일기준 데이터를 clone하여, 뉴스를 다시 다운로드 받은 날이 있다. 2019-10-22

In [15]:
275632+81

275713

In [10]:
from pathlib import Path
downloaded = [p.stem for p in Path('newsdata/downloaded').glob('**/*.json')]
trashed = [p.stem for p in Path('newsdata/trashed').glob('**/*.json')]

In [11]:
fnames_remove = set(downloaded) & set(trashed)

In [12]:
fnames_remove

set()

In [4]:
for fname in fnames_remove:
    file = Path('newsdata/trashed/' + fname[:3] + '/' + fname + '.json')
    file.unlink()